In [1]:
#pip install vowpalwabbit


#for cb problem you better need historical data

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
import os
import re

import pandas as pd
import numpy as np
import vowpalwabbit

from sklearn.preprocessing import LabelEncoder
from vowpalwabbit import Workspace

In [3]:
os.getcwd()

'C:\\Users\\Patrick Bormann\\Desktop\\Projekte\\Python\\Kursmaterial ML Engineer\\Projekt_3_Starbucks_Capstone\\Starbucks_submit'

In [4]:
data = pd.read_csv("./starbucks_data/starbucks_df_per_coupon_imputed.csv")

In [5]:
data

,id,event,coupon_code,time,value,became_member_on,reward,difficulty_offered,duration_offered,offer_type,email,social,web,mobile,offer_received,offer_viewed,offer_completed,transaction,offer_views_missed,informational,bogo,discount,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,total_reward_completed,total_duration_completed,total_difficulty_completed,amount_of_paths_completed_part1,amount_of_paths_completed_part2,amount_of_completed_user_paths,mean_time_between_actions,gender,income,age
0,0009655768c64bdeb2e877511632db8f,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,20170421,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0,1,0,0,0,1,1,0,0,168,0,168,0,0.0,0.0,0.0,0,0,0,0.0,M,72000.000000,33.0
1,0009655768c64bdeb2e877511632db8f,offer viewed,5a8bc65990b245e5a138643cd4eb9837,192,0.00,20170421,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0,0,1,0,0,-1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,24.0,M,72000.000000,33.0
2,0009655768c64bdeb2e877511632db8f,transaction,0,228,22.16,20170421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,36.0,M,72000.000000,33.0
3,0009655768c64bdeb2e877511632db8f,offer received,3f207df678b143eea3cee63160fa8bed,336,0.00,20170421,0.0,0.0,4.0,informational,1.0,0.0,1.0,1.0,1,0,0,0,1,1,0,0,336,0,336,0,0.0,0.0,0.0,0,0,0,108.0,M,72000.000000,33.0
4,0009655768c64bdeb2e877511632db8f,offer viewed,3f207df678b143eea3cee63160fa8bed,372,0.00,20170421,0.0,0.0,4.0,informational,1.0,0.0,1.0,1.0,0,1,0,0,-1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,36.0,M,72000.000000,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306529,ffff82501cea40309d5fdd7edcca4a07,offer received,2906b810c7d4411798c6938adc9daaa5,576,0.00,20161125,2.0,10.0,7.0,discount,1.0,0.0,1.0,1.0,1,0,0,0,1,0,0,1,576,0,576,0,0.0,0.0,0.0,0,0,0,0.0,M,47643.385261,46.0
306530,ffff82501cea40309d5fdd7edcca4a07,offer completed,2906b810c7d4411798c6938adc9daaa5,576,0.00,20161125,2.0,10.0,7.0,discount,1.0,0.0,1.0,1.0,0,0,1,0,0,0,0,1,0,576,0,576,2.0,7.0,10.0,0,0,0,0.0,M,47643.385261,46.0
306531,ffff82501cea40309d5fdd7edcca4a07,offer viewed,2906b810c7d4411798c6938adc9daaa5,582,0.00,20161125,2.0,10.0,7.0,discount,1.0,0.0,1.0,1.0,0,1,0,0,-1,0,0,1,0,0,0,0,0.0,0.0,0.0,0,0,0,6.0,M,47643.385261,46.0
306532,ffff82501cea40309d5fdd7edcca4a07,transaction,0,606,10.12,20161125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,24.0,M,47643.385261,46.0


In [6]:
data.coupon_code.unique() # actions

array(['5a8bc65990b245e5a138643cd4eb9837', '0',
       '3f207df678b143eea3cee63160fa8bed',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       'fafdcd668e3743c1bb461111dcafc2a4',
       '2906b810c7d4411798c6938adc9daaa5',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       '0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '9b98b8c7a33c4b65b9aebfe6a799e6d9',
       'ae264e3637204a6fb9bb56bc8210ddfd',
       '4d5c57ea9a6940dd891ad53e9dbe8da0'], dtype=object)

In [7]:
# -creating data format (OK)
# -probas by dividing amount of completed user paths or relate it co a coupon to calculate probability
# recode costs for amount of completed user paths (OK)
# split data to train with cluster == 2
# give features from cluster == 1

#users have to be aware of the coupn thus they first have to view it.

In [8]:
data[data.id=="0011e0d4e6b944f998e987f904e8c1e5"].sort_values("time")  #there are still bugs recognizing the correct path of users # e.g. mot counts if users viewed at two coupons or has two open coupons and fulfills two at once. its only applicable for one at a time coupons
#and this example shows that the user fulfilled two couponsnthat were applicable for completed user journeys, but only one of two was counted

,id,event,coupon_code,time,value,became_member_on,reward,difficulty_offered,duration_offered,offer_type,email,social,web,mobile,offer_received,offer_viewed,offer_completed,transaction,offer_views_missed,informational,bogo,discount,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,total_reward_completed,total_duration_completed,total_difficulty_completed,amount_of_paths_completed_part1,amount_of_paths_completed_part2,amount_of_completed_user_paths,mean_time_between_actions,gender,income,age
27,0011e0d4e6b944f998e987f904e8c1e5,offer received,3f207df678b143eea3cee63160fa8bed,0,0.00,20180109,0.0,0.0,4.0,informational,1.0,0.0,1.0,1.0,1,0,0,0,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0.0,F,90000.0,59.0
28,0011e0d4e6b944f998e987f904e8c1e5,offer viewed,3f207df678b143eea3cee63160fa8bed,6,0.00,20180109,0.0,0.0,4.0,informational,1.0,0.0,1.0,1.0,0,1,0,0,-1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,6.0,F,90000.0,59.0
29,0011e0d4e6b944f998e987f904e8c1e5,transaction,0,132,13.49,20180109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,126.0,F,90000.0,59.0
30,0011e0d4e6b944f998e987f904e8c1e5,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,168,0.00,20180109,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,1,0,0,0,1,0,0,1,168,0,168,0,0.0,0.0,0.0,1,0,1,36.0,F,90000.0,59.0
31,0011e0d4e6b944f998e987f904e8c1e5,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,186,0.00,20180109,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,0,1,0,0,-1,0,0,1,0,0,0,0,0.0,0.0,0.0,0,0,0,18.0,F,90000.0,59.0
32,0011e0d4e6b944f998e987f904e8c1e5,transaction,0,252,11.93,20180109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,66.0,F,90000.0,59.0
33,0011e0d4e6b944f998e987f904e8c1e5,offer completed,2298d6c36e964ae4a3e7e9706d1fb8c2,252,0.00,20180109,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,0,0,1,0,0,0,0,1,0,252,0,252,3.0,7.0,7.0,0,0,0,0.0,F,90000.0,59.0
34,0011e0d4e6b944f998e987f904e8c1e5,offer received,5a8bc65990b245e5a138643cd4eb9837,336,0.00,20180109,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0,1,0,0,0,1,1,0,0,336,0,336,0,0.0,0.0,0.0,0,0,0,84.0,F,90000.0,59.0
35,0011e0d4e6b944f998e987f904e8c1e5,offer viewed,5a8bc65990b245e5a138643cd4eb9837,354,0.00,20180109,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0,0,1,0,0,-1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,18.0,F,90000.0,59.0
36,0011e0d4e6b944f998e987f904e8c1e5,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,0.00,20180109,5.0,20.0,10.0,discount,1.0,0.0,1.0,0.0,1,0,0,0,1,0,0,1,408,0,408,0,0.0,0.0,0.0,0,0,0,54.0,F,90000.0,59.0


In [9]:
data[data.id=="fffad4f4828548d1b5583907f2e9906b"].sort_values("time") 

,id,event,coupon_code,time,value,became_member_on,reward,difficulty_offered,duration_offered,offer_type,email,social,web,mobile,offer_received,offer_viewed,offer_completed,transaction,offer_views_missed,informational,bogo,discount,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,total_reward_completed,total_duration_completed,total_difficulty_completed,amount_of_paths_completed_part1,amount_of_paths_completed_part2,amount_of_completed_user_paths,mean_time_between_actions,gender,income,age
306478,fffad4f4828548d1b5583907f2e9906b,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0,0.00,20170123,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0,1,0,0,0,1,0,1,0,0,0,0,0,0.0,0.0,0.0,1,0,1,0.0,M,53399.166667,54.0
306479,fffad4f4828548d1b5583907f2e9906b,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,6,0.00,20170123,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0,0,1,0,0,-1,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0,6.0,M,53399.166667,54.0
306480,fffad4f4828548d1b5583907f2e9906b,transaction,0,36,6.97,20170123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,30.0,M,53399.166667,54.0
306481,fffad4f4828548d1b5583907f2e9906b,offer completed,f19421c1d4aa40978ebb69ca19b0e20d,36,0.00,20170123,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,0,36,0,36,5.0,5.0,5.0,0,0,0,0.0,M,53399.166667,54.0
306482,fffad4f4828548d1b5583907f2e9906b,transaction,0,66,2.95,20170123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,30.0,M,53399.166667,54.0
306483,fffad4f4828548d1b5583907f2e9906b,transaction,0,84,10.44,20170123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,18.0,M,53399.166667,54.0
306484,fffad4f4828548d1b5583907f2e9906b,transaction,0,120,6.26,20170123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,36.0,M,53399.166667,54.0
306485,fffad4f4828548d1b5583907f2e9906b,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,20170123,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0,1,0,0,0,1,1,0,0,168,0,168,0,0.0,0.0,0.0,0,0,0,48.0,M,53399.166667,54.0
306486,fffad4f4828548d1b5583907f2e9906b,offer viewed,5a8bc65990b245e5a138643cd4eb9837,168,0.00,20170123,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0,0,1,0,0,-1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0.0,M,53399.166667,54.0
306487,fffad4f4828548d1b5583907f2e9906b,transaction,0,198,5.63,20170123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,30.0,M,53399.166667,54.0


In [10]:
#since we know knew that it works, we filter out tansactionsa s they are no coupons and although actions, but from the user, and not to choose from as an advertiser
data = data[data.coupon_code != "0"]

In [11]:
data["gender"] = np.where(data.gender == "M", 0, np.where(data.gender=="F", 1, 2))

<ipython-input-11-a01af2be9417>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gender"] = np.where(data.gender == "M", 0, np.where(data.gender=="F", 1, 2))


In [12]:
data.dtypes

id                                  object
event                               object
coupon_code                         object
time                                 int64
value                              float64
became_member_on                     int64
reward                             float64
difficulty_offered                 float64
duration_offered                   float64
offer_type                          object
email                              float64
social                             float64
web                                float64
mobile                             float64
offer_received                       int64
offer_viewed                         int64
offer_completed                      int64
transaction                          int64
offer_views_missed                   int64
informational                        int64
bogo                                 int64
discount                             int64
last_time_offer_received             int64
last_time_o

In [13]:
data_for_VW = data.groupby(["id", "coupon_code"]).agg(
{
     "value": "max",
     "transaction": "max",
     "gender": "max",
     "age": "max",
     "became_member_on": "max",
     "income": "max",
     "reward": "max", #
     "offer_views_missed": "max", #
     "informational": "max",#
     "bogo": "max",#
     "discount": "max",#
     "total_reward_completed": "max",
     "total_duration_completed": "max",
     "total_difficulty_completed": "max",
     "difficulty_offered": "max",
     "duration_offered": "max",
     "last_time_offer_received": "max",
     "last_time_offer_completed": "max",
     "first_time_offer_received": "min",
     "first_time_offer_completed": "min",
     "mean_time_between_actions": "mean",
     "amount_of_completed_user_paths": "max", #
     "email": "max",
     "social": "max",
     "web": "max",
     "mobile": "max"
    }
)

In [14]:
data_for_VW

value  \
id                               coupon_code                               
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5    0.0   
                                 3f207df678b143eea3cee63160fa8bed    0.0   
                                 5a8bc65990b245e5a138643cd4eb9837    0.0   
                                 f19421c1d4aa40978ebb69ca19b0e20d    0.0   
                                 fafdcd668e3743c1bb461111dcafc2a4    0.0   
...                                                                  ...   
fffad4f4828548d1b5583907f2e9906b f19421c1d4aa40978ebb69ca19b0e20d    0.0   
ffff82501cea40309d5fdd7edcca4a07 0b1e1539f2cc45b7b9fa7c272da2e1d7    0.0   
                                 2906b810c7d4411798c6938adc9daaa5    0.0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9    0.0   
                                 fafdcd668e3743c1bb461111dcafc2a4    0.0   

                                                                   transaction  \
id                               coupon_code                                     
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5            0   
                                 3f207df678b143eea3cee63160fa8bed            0   
                                 5a8bc65990b245e5a138643cd4eb9837            0   
                                 f19421c1d4aa40978ebb69ca19b0e20d            0   
                                 fafdcd668e3743c1bb461111dcafc2a4            0   
...                                                                        ...   
fffad4f4828548d1b5583907f2e9906b f19421c1d4aa40978ebb69ca19b0e20d            0   
ffff82501cea40309d5fdd7edcca4a07 0b1e1539f2cc45b7b9fa7c272da2e1d7            0   
                                 2906b810c7d4411798c6938adc9daaa5            0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            0   
                                 fafdcd668e3743c1bb461111dcafc2a4            0   

                                                                   gender  \
id                               coupon_code                                
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5       0   
                                 3f207df678b143eea3cee63160fa8bed       0   
                                 5a8bc65990b245e5a138643cd4eb9837       0   
                                 f19421c1d4aa40978ebb69ca19b0e20d       0   
                                 fafdcd668e3743c1bb461111dcafc2a4       0   
...                                                                   ...   
fffad4f4828548d1b5583907f2e9906b f19421c1d4aa40978ebb69ca19b0e20d       0   
ffff82501cea40309d5fdd7edcca4a07 0b1e1539f2cc45b7b9fa7c272da2e1d7       0   
                                 2906b810c7d4411798c6938adc9daaa5       0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9       0   
                                 fafdcd668e3743c1bb461111dcafc2a4       0   

                                                                    age  \
id                               coupon_code                              
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5  33.0   
                                 3f207df678b143eea3cee63160fa8bed  33.0   
                                 5a8bc65990b245e5a138643cd4eb9837  33.0   
                                 f19421c1d4aa40978ebb69ca19b0e20d  33.0   
                                 fafdcd668e3743c1bb461111dcafc2a4  33.0   
...                                                                 ...   
fffad4f4828548d1b5583907f2e9906b f19421c1d4aa40978ebb69ca19b0e20d  54.0   
ffff82501cea40309d5fdd7edcca4a07 0b1e1539f2cc45b7b9fa7c272da2e1d7  46.0   
                                 2906b810c7d4411798c6938adc9daaa5  46.0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9  46.0   
                                 fafdcd668e3743c1bb461111dcafc2a4  46.0   

                                 

In [15]:
# bring data into historical order per user
data_for_VW = data_for_VW.reset_index().sort_values(
    ["id", 
     "last_time_offer_received"
    ]
)

In [16]:
data_for_VW[data_for_VW.id =="fffad4f4828548d1b5583907f2e9906b"]

,id,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile
63281,fffad4f4828548d1b5583907f2e9906b,5a8bc65990b245e5a138643cd4eb9837,0.0,0,0,54.0,20170123,53399.166667,0.0,1,1,0,0,0.0,0.0,0.0,0.0,3.0,168,0,0,0,24.0,0,1.0,1.0,0.0,1.0
63283,fffad4f4828548d1b5583907f2e9906b,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0,0,54.0,20170123,53399.166667,5.0,1,0,1,0,5.0,5.0,5.0,5.0,5.0,408,516,0,0,19.0,1,1.0,1.0,1.0,1.0
63282,fffad4f4828548d1b5583907f2e9906b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,54.0,20170123,53399.166667,5.0,1,0,1,0,5.0,7.0,5.0,5.0,7.0,576,588,0,0,32.0,0,1.0,0.0,1.0,1.0


In [17]:
#let the value of a user action increase in value over time
cumsum_list = [
    "value"
    "transaction",    
    "offer_views_missed", 
    "informational",
    "bogo",
    "discount",
    "total_reward_completed",
    "total_duration_completed",
    "total_difficulty_completed",
    "difficulty_offered",
    "duration_offered",
    "email",
    "social",
    "web",
    "mobile"
] #cumsum per key needed
data_for_VW_sum = data_for_VW.apply(lambda x: data_for_VW.groupby("id")[x.name].cumsum() if x.name in cumsum_list else x)

In [18]:
data_for_VW_sum

,id,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,0.0,0,0,33.0,20170421,72000.000000,0.0,1,1,0,0,0.0,0.0,0.0,0.0,3.0,168,0,0,0,12.0,0,1.0,1.0,0.0,1.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,0.0,0,0,33.0,20170421,72000.000000,0.0,2,2,0,0,0.0,0.0,0.0,0.0,7.0,336,0,0,0,72.0,0,2.0,1.0,1.0,2.0
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0,0,33.0,20170421,72000.000000,5.0,3,2,1,0,5.0,5.0,5.0,5.0,12.0,408,414,0,0,26.0,0,3.0,2.0,2.0,3.0
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,0,33.0,20170421,72000.000000,2.0,4,2,1,1,7.0,15.0,15.0,15.0,22.0,504,528,0,0,20.0,0,4.0,3.0,3.0,4.0
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.0,0,0,33.0,20170421,72000.000000,2.0,5,2,1,2,9.0,22.0,25.0,25.0,29.0,576,576,0,0,0.0,0,5.0,3.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63282,fffad4f4828548d1b5583907f2e9906b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,54.0,20170123,53399.166667,5.0,3,1,2,0,10.0,12.0,10.0,10.0,15.0,576,588,0,0,32.0,0,3.0,2.0,2.0,3.0
63287,ffff82501cea40309d5fdd7edcca4a07,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,0,46.0,20161125,47643.385261,2.0,1,0,0,1,2.0,10.0,10.0,10.0,10.0,0,60,0,0,2.0,1,1.0,1.0,1.0,1.0
63284,ffff82501cea40309d5fdd7edcca4a07,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0,0,46.0,20161125,47643.385261,5.0,2,0,0,2,7.0,20.0,30.0,30.0,20.0,168,198,0,0,18.0,1,2.0,1.0,2.0,1.0
63286,ffff82501cea40309d5fdd7edcca4a07,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,46.0,20161125,47643.385261,5.0,3,0,1,2,12.0,27.0,35.0,35.0,27.0,504,504,0,0,10.0,0,3.0,1.0,3.0,2.0


In [19]:
data_for_VW_sum[data_for_VW_sum.id=="fffad4f4828548d1b5583907f2e9906b"] #really had two for example

,id,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile
63281,fffad4f4828548d1b5583907f2e9906b,5a8bc65990b245e5a138643cd4eb9837,0.0,0,0,54.0,20170123,53399.166667,0.0,1,1,0,0,0.0,0.0,0.0,0.0,3.0,168,0,0,0,24.0,0,1.0,1.0,0.0,1.0
63283,fffad4f4828548d1b5583907f2e9906b,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0,0,54.0,20170123,53399.166667,5.0,2,1,1,0,5.0,5.0,5.0,5.0,8.0,408,516,0,0,19.0,1,2.0,2.0,1.0,2.0
63282,fffad4f4828548d1b5583907f2e9906b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,54.0,20170123,53399.166667,5.0,3,1,2,0,10.0,12.0,10.0,10.0,15.0,576,588,0,0,32.0,0,3.0,2.0,2.0,3.0


In [20]:
data_for_VW_sum

,id,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,0.0,0,0,33.0,20170421,72000.000000,0.0,1,1,0,0,0.0,0.0,0.0,0.0,3.0,168,0,0,0,12.0,0,1.0,1.0,0.0,1.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,0.0,0,0,33.0,20170421,72000.000000,0.0,2,2,0,0,0.0,0.0,0.0,0.0,7.0,336,0,0,0,72.0,0,2.0,1.0,1.0,2.0
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0,0,33.0,20170421,72000.000000,5.0,3,2,1,0,5.0,5.0,5.0,5.0,12.0,408,414,0,0,26.0,0,3.0,2.0,2.0,3.0
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,0,33.0,20170421,72000.000000,2.0,4,2,1,1,7.0,15.0,15.0,15.0,22.0,504,528,0,0,20.0,0,4.0,3.0,3.0,4.0
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.0,0,0,33.0,20170421,72000.000000,2.0,5,2,1,2,9.0,22.0,25.0,25.0,29.0,576,576,0,0,0.0,0,5.0,3.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63282,fffad4f4828548d1b5583907f2e9906b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,54.0,20170123,53399.166667,5.0,3,1,2,0,10.0,12.0,10.0,10.0,15.0,576,588,0,0,32.0,0,3.0,2.0,2.0,3.0
63287,ffff82501cea40309d5fdd7edcca4a07,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,0,46.0,20161125,47643.385261,2.0,1,0,0,1,2.0,10.0,10.0,10.0,10.0,0,60,0,0,2.0,1,1.0,1.0,1.0,1.0
63284,ffff82501cea40309d5fdd7edcca4a07,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0,0,46.0,20161125,47643.385261,5.0,2,0,0,2,7.0,20.0,30.0,30.0,20.0,168,198,0,0,18.0,1,2.0,1.0,2.0,1.0
63286,ffff82501cea40309d5fdd7edcca4a07,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,46.0,20161125,47643.385261,5.0,3,0,1,2,12.0,27.0,35.0,35.0,27.0,504,504,0,0,10.0,0,3.0,1.0,3.0,2.0


In [21]:
data_for_VW_sum.groupby("id").amount_of_completed_user_paths.max()

id
0009655768c64bdeb2e877511632db8f    0
00116118485d4dfda04fdbaba9a87b5c    0
0011e0d4e6b944f998e987f904e8c1e5    1
0020c2b971eb4e9188eac86d93036a77    1
0020ccbbb6d84e358d3414a3ff76cffd    1
                                   ..
fff3ba4757bd42088c044ca26d73817a    0
fff7576017104bcc8677a8d63322b5e1    1
fff8957ea8b240a6b5e634b6ee8eafcf    0
fffad4f4828548d1b5583907f2e9906b    1
ffff82501cea40309d5fdd7edcca4a07    1
Name: amount_of_completed_user_paths, Length: 16994, dtype: int64

In [22]:
#add flag for max amount of user paths for clustering
#for id in data_for_VW.id.unique()[:10]:
    #if data_for_VW[data_for_VW.id == id].amount_of_completed_user_paths.max() == 0
cluster_list_for_ids = np.where(data_for_VW_sum.groupby("id").amount_of_completed_user_paths.sum() == 0, 0, np.where(data_for_VW_sum.groupby("id").amount_of_completed_user_paths.sum() == 1, 1, 2))

In [23]:
print(len(data_for_VW_sum.id.unique()))
print(len(cluster_list_for_ids))

16994
16994


In [24]:
data_for_VW_sum.id.unique()

array(['0009655768c64bdeb2e877511632db8f',
       '00116118485d4dfda04fdbaba9a87b5c',
       '0011e0d4e6b944f998e987f904e8c1e5', ...,
       'fff8957ea8b240a6b5e634b6ee8eafcf',
       'fffad4f4828548d1b5583907f2e9906b',
       'ffff82501cea40309d5fdd7edcca4a07'], dtype=object)

In [25]:
match_clusters_frame = pd.DataFrame({
  "id":data_for_VW_sum.id.unique(),
   "cluster": cluster_list_for_ids
})

In [26]:
match_clusters_frame

,id,cluster
0,0009655768c64bdeb2e877511632db8f,0
1,00116118485d4dfda04fdbaba9a87b5c,0
2,0011e0d4e6b944f998e987f904e8c1e5,2
3,0020c2b971eb4e9188eac86d93036a77,1
4,0020ccbbb6d84e358d3414a3ff76cffd,2
...,...,...
16989,fff3ba4757bd42088c044ca26d73817a,0
16990,fff7576017104bcc8677a8d63322b5e1,1
16991,fff8957ea8b240a6b5e634b6ee8eafcf,0
16992,fffad4f4828548d1b5583907f2e9906b,1


In [27]:
unique, counts = np.unique(np.array(cluster_list_for_ids), return_counts=True)

In [28]:
counts #counts users not actions

array([8600, 4527, 3867], dtype=int64)

In [29]:
data_for_VW_sum.set_index("id", inplace=True)
match_clusters_frame.set_index("id", inplace=True)

In [30]:
data_for_VW_sum = data_for_VW_sum.join(match_clusters_frame, on="id")

In [31]:
#adding aws clusters

In [32]:
data_for_VW_sum

,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile,cluster
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,0.0,0,0,33.0,20170421,72000.000000,0.0,1,1,0,0,0.0,0.0,0.0,0.0,3.0,168,0,0,0,12.0,0,1.0,1.0,0.0,1.0,0
0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,0.0,0,0,33.0,20170421,72000.000000,0.0,2,2,0,0,0.0,0.0,0.0,0.0,7.0,336,0,0,0,72.0,0,2.0,1.0,1.0,2.0,0
0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0,0,33.0,20170421,72000.000000,5.0,3,2,1,0,5.0,5.0,5.0,5.0,12.0,408,414,0,0,26.0,0,3.0,2.0,2.0,3.0,0
0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,0,33.0,20170421,72000.000000,2.0,4,2,1,1,7.0,15.0,15.0,15.0,22.0,504,528,0,0,20.0,0,4.0,3.0,3.0,4.0,0
0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.0,0,0,33.0,20170421,72000.000000,2.0,5,2,1,2,9.0,22.0,25.0,25.0,29.0,576,576,0,0,0.0,0,5.0,3.0,4.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffad4f4828548d1b5583907f2e9906b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,0,54.0,20170123,53399.166667,5.0,3,1,2,0,10.0,12.0,10.0,10.0,15.0,576,588,0,0,32.0,0,3.0,2.0,2.0,3.0,1
ffff82501cea40309d5fdd7edcca4a07,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,0,46.0,20161125,47643.385261,2.0,1,0,0,1,2.0,10.0,10.0,10.0,10.0,0,60,0,0,2.0,1,1.0,1.0,1.0,1.0,2
ffff82501cea40309d5fdd7edcca4a07,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0,0,46.0,20161125,47643.385261,5.0,2,0,0,2,7.0,20.0,30.0,30.0,20.0,168,198,0,0,18.0,1,2.0,1.0,2.0,1.0,2


In [33]:
#encoding the target
#data_for_VW.amount_of_completed_user_paths = np.where(data_for_VW.amount_of_completed_user_paths >=1, 1, 0)

In [34]:
data_for_VW_sum[data_for_VW_sum.index.get_level_values(0)=="0011e0d4e6b944f998e987f904e8c1e5"] #really had two full journeys for example

,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile,cluster
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,0.0,0,1,59.0,20180109,90000.0,0.0,1,1,0,0,0.0,0.0,0.0,0.0,4.0,0,0,0,0,3.0,0,1.0,0.0,1.0,1.0,2
0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,0.0,0,1,59.0,20180109,90000.0,3.0,2,1,0,1,3.0,7.0,7.0,7.0,11.0,168,252,0,0,18.0,1,2.0,1.0,2.0,2.0,2
0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,0.0,0,1,59.0,20180109,90000.0,0.0,3,2,0,1,3.0,7.0,7.0,7.0,14.0,336,0,0,0,51.0,0,3.0,2.0,2.0,3.0,2
0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0,1,59.0,20180109,90000.0,5.0,4,2,0,2,8.0,17.0,27.0,27.0,24.0,408,576,0,0,26.0,0,4.0,2.0,3.0,3.0,2
0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,1,59.0,20180109,90000.0,5.0,5,2,1,2,13.0,24.0,32.0,32.0,31.0,504,576,0,0,28.0,1,5.0,2.0,4.0,4.0,2


In [35]:
#Are there user who got a coupon twice (and if yes, really fulfilled the user journey)?
for user_id in data_for_VW.index.unique():
    data = data_for_VW[data_for_VW.index ==user_id].coupon_code.duplicated(keep=False)
    
    if data.sum() > 0:
        print(data)

In [36]:
#encoding the probability
proba_data = data_for_VW_sum.groupby("id").agg({"amount_of_completed_user_paths": "sum"}).reset_index().rename(columns={'amount_of_completed_user_paths': "denominator"}).set_index("id")
proba_data.index

Index(['0009655768c64bdeb2e877511632db8f', '00116118485d4dfda04fdbaba9a87b5c',
       '0011e0d4e6b944f998e987f904e8c1e5', '0020c2b971eb4e9188eac86d93036a77',
       '0020ccbbb6d84e358d3414a3ff76cffd', '003d66b6608740288d6cc97a6903f4f0',
       '00426fe3ffde4c6b9cb9ad6d077a13ea', '004b041fbfe44859945daa2c7f79ee64',
       '004c5799adbf42868b9cff0396190900', '005500a7188546ff8a767329a2f7c76a',
       ...
       'ffecb1f8543f4bf7bade023de366d6bf', 'ffed75d3abc64b488982f50ed12878b5',
       'ffede3b700ac41d6a266fa1ba74b4f16', 'fff0f0aac6c547b9b263080f09a5586a',
       'fff29fb549084123bd046dbc5ceb4faa', 'fff3ba4757bd42088c044ca26d73817a',
       'fff7576017104bcc8677a8d63322b5e1', 'fff8957ea8b240a6b5e634b6ee8eafcf',
       'fffad4f4828548d1b5583907f2e9906b', 'ffff82501cea40309d5fdd7edcca4a07'],
      dtype='object', name='id', length=16994)

In [37]:
#denominator 3 correct?
data_for_VW_sum[data_for_VW_sum.index == "0020ccbbb6d84e358d3414a3ff76cffd"]

,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile,cluster
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0020ccbbb6d84e358d3414a3ff76cffd,2298d6c36e964ae4a3e7e9706d1fb8c2,0.0,0,1,26.0,20161111,73000.0,3.0,1,0,0,1,3.0,7.0,7.0,7.0,7.0,168,222,0,0,42.0,1,1.0,1.0,1.0,1.0,2
0020ccbbb6d84e358d3414a3ff76cffd,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0,1,26.0,20161111,73000.0,5.0,2,0,1,1,8.0,12.0,12.0,12.0,12.0,336,378,0,0,16.0,1,2.0,2.0,2.0,2.0,2
0020ccbbb6d84e358d3414a3ff76cffd,5a8bc65990b245e5a138643cd4eb9837,0.0,0,1,26.0,20161111,73000.0,0.0,3,1,1,1,8.0,12.0,12.0,12.0,15.0,408,0,0,0,15.0,0,3.0,3.0,2.0,3.0,2
0020ccbbb6d84e358d3414a3ff76cffd,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0,1,26.0,20161111,73000.0,5.0,4,1,2,1,13.0,19.0,17.0,17.0,22.0,504,600,0,0,40.0,1,4.0,3.0,3.0,4.0,2


In [38]:
data_for_VW_sum.reset_index(inplace=True)
data_for_VW_sum.set_index("id", inplace=True)

In [39]:
all_data = data_for_VW_sum.merge(proba_data, on="id")

In [40]:
lbl = LabelEncoder()
coupons_groups = all_data
coupons_groups["Encoding"] = lbl.fit_transform(all_data.coupon_code)
coupons_groups = coupons_groups[["coupon_code", "Encoding"]]

all_data["coupon_code"] = lbl.fit_transform(all_data.coupon_code)

In [41]:
all_data["proba"] = all_data.amount_of_completed_user_paths/all_data.denominator

In [42]:
all_data.fillna(0, inplace=True)

In [43]:
all_data

,coupon_code,value,transaction,gender,age,became_member_on,income,reward,offer_views_missed,informational,bogo,discount,total_reward_completed,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,amount_of_completed_user_paths,email,social,web,mobile,cluster,denominator,Encoding,proba
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,5,0.0,0,0,33.0,20170421,72000.000000,0.0,1,1,0,0,0.0,0.0,0.0,0.0,3.0,168,0,0,0,12.0,0,1.0,1.0,0.0,1.0,0,0,5,0.000000
0009655768c64bdeb2e877511632db8f,3,0.0,0,0,33.0,20170421,72000.000000,0.0,2,2,0,0,0.0,0.0,0.0,0.0,7.0,336,0,0,0,72.0,0,2.0,1.0,1.0,2.0,0,0,3,0.000000
0009655768c64bdeb2e877511632db8f,8,0.0,0,0,33.0,20170421,72000.000000,5.0,3,2,1,0,5.0,5.0,5.0,5.0,12.0,408,414,0,0,26.0,0,3.0,2.0,2.0,3.0,0,0,8,0.000000
0009655768c64bdeb2e877511632db8f,9,0.0,0,0,33.0,20170421,72000.000000,2.0,4,2,1,1,7.0,15.0,15.0,15.0,22.0,504,528,0,0,20.0,0,4.0,3.0,3.0,4.0,0,0,9,0.000000
0009655768c64bdeb2e877511632db8f,2,0.0,0,0,33.0,20170421,72000.000000,2.0,5,2,1,2,9.0,22.0,25.0,25.0,29.0,576,576,0,0,0.0,0,5.0,3.0,4.0,5.0,0,0,2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffad4f4828548d1b5583907f2e9906b,6,0.0,0,0,54.0,20170123,53399.166667,5.0,3,1,2,0,10.0,12.0,10.0,10.0,15.0,576,588,0,0,32.0,0,3.0,2.0,2.0,3.0,1,1,6,0.000000
ffff82501cea40309d5fdd7edcca4a07,9,0.0,0,0,46.0,20161125,47643.385261,2.0,1,0,0,1,2.0,10.0,10.0,10.0,10.0,0,60,0,0,2.0,1,1.0,1.0,1.0,1.0,2,3,9,0.333333
ffff82501cea40309d5fdd7edcca4a07,0,0.0,0,0,46.0,20161125,47643.385261,5.0,2,0,0,2,7.0,20.0,30.0,30.0,20.0,168,198,0,0,18.0,1,2.0,1.0,2.0,1.0,2,3,0,0.333333


In [44]:
# create cost for business for completing the user path or for not fulfilling the user path? give the user something that increases converting probability but not has too high costs, otherwise the bandit would always advise the coupon with the highest reward
all_data["cost"] = all_data["reward"] #cost for business is the reward of the user for completing the journey
#cost can be the reward or the negative of the amount of completed user paths ?

In [45]:
#rename the other parts of the frame for the cb problem

In [46]:
all_data.rename(columns={
    "coupon_code": "action",
    "proba": "probability",
}, inplace=True)

In [47]:
# reorganize data frame

In [48]:
list(all_data.columns)

['action',
 'value',
 'transaction',
 'gender',
 'age',
 'became_member_on',
 'income',
 'reward',
 'offer_views_missed',
 'informational',
 'bogo',
 'discount',
 'total_reward_completed',
 'total_duration_completed',
 'total_difficulty_completed',
 'difficulty_offered',
 'duration_offered',
 'last_time_offer_received',
 'last_time_offer_completed',
 'first_time_offer_received',
 'first_time_offer_completed',
 'mean_time_between_actions',
 'amount_of_completed_user_paths',
 'email',
 'social',
 'web',
 'mobile',
 'cluster',
 'denominator',
 'Encoding',
 'probability',
 'cost']

In [49]:
all_data = all_data[[
    "action",
    "cost",
    "probability",
    'value',
    'gender',
    'age',
    'became_member_on',
    'income',
    'offer_views_missed',
    'informational',
    'bogo',
    'discount',
    'total_duration_completed',
    'total_difficulty_completed',
    'difficulty_offered',
    'duration_offered',
    'last_time_offer_received',
    'last_time_offer_completed',
    'first_time_offer_received',
    'first_time_offer_completed',
    'mean_time_between_actions',
    'email',
    'social',
    'web',
    'mobile',
    'cluster'
]]

#drop amount of user paths at it is partly correlated in proba

In [50]:
all_data.cluster.unique()

array([0, 2, 1])

In [51]:
all_data

,action,cost,probability,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile,cluster
id,,,,,,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,5,0.0,0.000000,0.0,0,33.0,20170421,72000.000000,1,1,0,0,0.0,0.0,0.0,3.0,168,0,0,0,12.0,1.0,1.0,0.0,1.0,0
0009655768c64bdeb2e877511632db8f,3,0.0,0.000000,0.0,0,33.0,20170421,72000.000000,2,2,0,0,0.0,0.0,0.0,7.0,336,0,0,0,72.0,2.0,1.0,1.0,2.0,0
0009655768c64bdeb2e877511632db8f,8,5.0,0.000000,0.0,0,33.0,20170421,72000.000000,3,2,1,0,5.0,5.0,5.0,12.0,408,414,0,0,26.0,3.0,2.0,2.0,3.0,0
0009655768c64bdeb2e877511632db8f,9,2.0,0.000000,0.0,0,33.0,20170421,72000.000000,4,2,1,1,15.0,15.0,15.0,22.0,504,528,0,0,20.0,4.0,3.0,3.0,4.0,0
0009655768c64bdeb2e877511632db8f,2,2.0,0.000000,0.0,0,33.0,20170421,72000.000000,5,2,1,2,22.0,25.0,25.0,29.0,576,576,0,0,0.0,5.0,3.0,4.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffad4f4828548d1b5583907f2e9906b,6,5.0,0.000000,0.0,0,54.0,20170123,53399.166667,3,1,2,0,12.0,10.0,10.0,15.0,576,588,0,0,32.0,3.0,2.0,2.0,3.0,1
ffff82501cea40309d5fdd7edcca4a07,9,2.0,0.333333,0.0,0,46.0,20161125,47643.385261,1,0,0,1,10.0,10.0,10.0,10.0,0,60,0,0,2.0,1.0,1.0,1.0,1.0,2
ffff82501cea40309d5fdd7edcca4a07,0,5.0,0.333333,0.0,0,46.0,20161125,47643.385261,2,0,0,2,20.0,30.0,30.0,20.0,168,198,0,0,18.0,2.0,1.0,2.0,1.0,2


In [52]:
#amount of user actions per cluster
print(len(all_data[all_data.cluster == 0]))
print(len(all_data[all_data.cluster == 1]))
print(len(all_data[all_data.cluster == 2]))

31301
16523
15464


In [53]:
print(len(all_data[all_data.cluster == 0].index.unique()))
print(len(all_data[all_data.cluster == 1].index.unique()))
print(len(all_data[all_data.cluster == 2].index.unique()))

8600
4527
3867


In [54]:
train_data = all_data[all_data.cluster == 2]

In [55]:
test_data = all_data[all_data.cluster == 1]

In [56]:
train_data.drop(columns=["cluster"], inplace=True)
test_data.drop(columns=["action", "cost", "probability", "cluster"], inplace=True)

<ipython-input-56-7bcbffc864fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(columns=["cluster"], inplace=True)
<ipython-input-56-7bcbffc864fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=["action", "cost", "probability", "cluster"], inplace=True)


In [57]:
train_data.index

Index(['0011e0d4e6b944f998e987f904e8c1e5', '0011e0d4e6b944f998e987f904e8c1e5',
       '0011e0d4e6b944f998e987f904e8c1e5', '0011e0d4e6b944f998e987f904e8c1e5',
       '0011e0d4e6b944f998e987f904e8c1e5', '0020ccbbb6d84e358d3414a3ff76cffd',
       '0020ccbbb6d84e358d3414a3ff76cffd', '0020ccbbb6d84e358d3414a3ff76cffd',
       '0020ccbbb6d84e358d3414a3ff76cffd', '004b041fbfe44859945daa2c7f79ee64',
       ...
       'ffdefcac307f4ca99ac1ebd51470f106', 'ffdefcac307f4ca99ac1ebd51470f106',
       'fff29fb549084123bd046dbc5ceb4faa', 'fff29fb549084123bd046dbc5ceb4faa',
       'fff29fb549084123bd046dbc5ceb4faa', 'fff29fb549084123bd046dbc5ceb4faa',
       'ffff82501cea40309d5fdd7edcca4a07', 'ffff82501cea40309d5fdd7edcca4a07',
       'ffff82501cea40309d5fdd7edcca4a07', 'ffff82501cea40309d5fdd7edcca4a07'],
      dtype='object', name='id', length=15464)

In [58]:
train_data.describe()

,action,cost,probability,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile
count,15464.000000,15464.000000,15464.000000,15464.0,15464.000000,15464.000000,1.546400e+04,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000,15464.0,15464.000000,15464.000000,15464.000000,15464.000000,15464.000000
mean,4.656622,4.408109,0.250065,0.0,0.389033,53.186498,2.016501e+07,63132.465019,2.599263,0.430678,1.109092,1.059493,14.334907,18.494633,20.319258,17.081932,354.588205,315.800828,15.426798,0.0,24.120958,2.599263,1.647504,2.105794,2.369309
std,2.917507,3.371973,0.223691,0.0,0.510865,15.723041,1.028340e+04,20978.670982,1.280027,0.570667,0.865118,0.856641,8.453740,11.874943,12.241726,8.819712,192.550467,238.375625,81.847606,0.0,23.764491,1.280027,1.041639,1.170376,1.233410
min,0.000000,0.000000,0.000000,0.0,0.000000,18.000000,2.013080e+07,30000.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,0.000000,0.0,0.000000,45.000000,2.016033e+07,47264.305556,1.000000,0.000000,0.000000,0.000000,7.000000,10.000000,10.000000,10.000000,168.000000,24.000000,0.000000,0.0,8.000000,1.000000,1.000000,1.000000,1.000000
50%,5.000000,5.000000,0.333333,0.0,0.000000,53.000000,2.017013e+07,59000.000000,2.000000,0.000000,1.000000,1.000000,14.000000,17.000000,20.000000,17.000000,408.000000,372.000000,0.000000,0.0,18.000000,2.000000,2.000000,2.000000,2.000000
75%,7.000000,5.000000,0.500000,0.0,1.000000,64.000000,2.017101e+07,76000.000000,4.000000,1.000000,2.000000,2.000000,20.000000,27.000000,30.000000,24.000000,504.000000,534.000000,0.000000,0.0,34.000000,4.000000,2.000000,3.000000,3.000000
max,9.000000,10.000000,0.500000,0.0,2.000000,90.000000,2.018073e+07,120000.000000,6.000000,2.000000,4.000000,4.000000,46.000000,62.000000,62.000000,46.000000,576.000000,714.000000,576.000000,0.0,402.000000,6.000000,5.000000,6.000000,6.000000


In [59]:
test_data.describe()

,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile
count,16523.0,16523.000000,16523.000000,1.652300e+04,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000,16523.0,16523.000000,16523.000000,16523.000000,16523.000000,16523.000000
mean,0.0,0.388247,53.237124,2.016673e+07,63572.085477,2.459178,0.515766,0.960056,0.983357,10.241058,13.189554,18.861042,15.955456,360.888943,249.109121,36.283968,0.0,30.993221,2.459178,1.455728,1.965987,2.198088
std,0.0,0.514433,15.528882,1.119751e+04,20749.020536,1.220245,0.614587,0.825380,0.830380,7.785835,11.075186,12.111866,8.454981,190.478111,253.306182,122.241993,0.0,31.145951,1.220245,0.981624,1.115443,1.170561
min,0.0,0.000000,18.000000,2.013073e+07,30000.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,45.000000,2.016052e+07,48000.000000,1.000000,0.000000,0.000000,0.000000,5.000000,5.000000,10.000000,10.000000,168.000000,0.000000,0.000000,0.0,9.000000,1.000000,1.000000,1.000000,1.000000
50%,0.0,0.000000,53.000000,2.017061e+07,60000.000000,2.000000,0.000000,1.000000,1.000000,10.000000,10.000000,17.000000,15.000000,408.000000,186.000000,0.000000,0.0,24.000000,2.000000,1.000000,2.000000,2.000000
75%,0.0,1.000000,64.000000,2.017121e+07,76000.000000,3.000000,1.000000,1.000000,1.000000,15.000000,20.000000,27.000000,22.000000,504.000000,504.000000,0.000000,0.0,42.000000,3.000000,2.000000,3.000000,3.000000
max,0.0,2.000000,90.000000,2.018073e+07,120000.000000,6.000000,2.000000,4.000000,4.000000,41.000000,60.000000,67.000000,46.000000,576.000000,714.000000,576.000000,0.0,408.000000,6.000000,5.000000,6.000000,6.000000


In [60]:
# "--cb_explore 10 --cover 3"
# ""--cb_explore_adf --bag 5"

In [61]:
vw = vowpalwabbit.Workspace("--cb_explore 10 --cover 3", quiet=True)  #cb_explore adf? https://vowpalwabbit.org/docs/vowpal_wabbit/python/latest/tutorials/python_Contextual_bandits_and_Vowpal_Wabbit.html#

In [62]:
for i in range(len(train_data)):
    action = train_data["action"].iloc[i]
    cost = train_data["cost"].iloc[i]
    probability = train_data["probability"].iloc[i]
    # Construct the example in the required vw format.
    #print(learn_example)
    learn_example = \
        str(action) + ":" + \
        str(cost) + ":" + \
        str(probability) + " | " + \
        ' '.join([str(row) for row in train_data.iloc[i, 3:].values])
    vw.learn(learn_example)
    #The policies differ by being trained on different random subsets of the data, with each example going to a subset of the m policies
    # because policy equals model we have a subset of data going to every model. 
   # vw.learn(learn_example)


In [63]:
train_data

,action,cost,probability,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile
id,,,,,,,,,,,,,,,,,,,,,,,,,
0011e0d4e6b944f998e987f904e8c1e5,3,0.0,0.000000,0.0,1,59.0,20180109,90000.000000,1,1,0,0,0.0,0.0,0.0,4.0,0,0,0,0,3.0,1.0,0.0,1.0,1.0
0011e0d4e6b944f998e987f904e8c1e5,1,3.0,0.500000,0.0,1,59.0,20180109,90000.000000,2,1,0,1,7.0,7.0,7.0,11.0,168,252,0,0,18.0,2.0,1.0,2.0,2.0
0011e0d4e6b944f998e987f904e8c1e5,5,0.0,0.000000,0.0,1,59.0,20180109,90000.000000,3,2,0,1,7.0,7.0,7.0,14.0,336,0,0,0,51.0,3.0,2.0,2.0,3.0
0011e0d4e6b944f998e987f904e8c1e5,0,5.0,0.000000,0.0,1,59.0,20180109,90000.000000,4,2,0,2,17.0,27.0,27.0,24.0,408,576,0,0,26.0,4.0,2.0,3.0,3.0
0011e0d4e6b944f998e987f904e8c1e5,6,5.0,0.500000,0.0,1,59.0,20180109,90000.000000,5,2,1,2,24.0,32.0,32.0,31.0,504,576,0,0,28.0,5.0,2.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff29fb549084123bd046dbc5ceb4faa,7,10.0,0.333333,0.0,0,56.0,20170831,64438.333333,4,0,3,1,27.0,35.0,35.0,27.0,576,684,0,0,14.4,4.0,4.0,3.0,4.0
ffff82501cea40309d5fdd7edcca4a07,9,2.0,0.333333,0.0,0,46.0,20161125,47643.385261,1,0,0,1,10.0,10.0,10.0,10.0,0,60,0,0,2.0,1.0,1.0,1.0,1.0
ffff82501cea40309d5fdd7edcca4a07,0,5.0,0.333333,0.0,0,46.0,20161125,47643.385261,2,0,0,2,20.0,30.0,30.0,20.0,168,198,0,0,18.0,2.0,1.0,2.0,1.0


In [64]:
test_data

,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile
id,,,,,,,,,,,,,,,,,,,,,,
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,1,0,1,0,0.0,0.0,10.0,7.0,168,0,168,0,24.0,1.0,1.0,0.0,1.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,2,0,1,1,10.0,10.0,20.0,17.0,336,510,0,0,36.0,2.0,2.0,1.0,2.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,3,0,2,1,15.0,20.0,30.0,22.0,408,510,0,0,30.0,3.0,3.0,2.0,3.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,4,1,2,1,15.0,20.0,30.0,25.0,504,0,0,0,99.0,4.0,4.0,2.0,4.0
005500a7188546ff8a767329a2f7c76a,0.0,0,54.0,20171209,91000.000000,1,0,0,1,0.0,0.0,10.0,7.0,408,0,408,0,144.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,3,0,2,1,10.0,10.0,30.0,22.0,504,594,0,0,22.0,3.0,3.0,2.0,3.0
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,4,0,3,1,17.0,15.0,35.0,29.0,576,594,0,0,9.0,4.0,3.0,3.0,4.0
fffad4f4828548d1b5583907f2e9906b,0.0,0,54.0,20170123,53399.166667,1,1,0,0,0.0,0.0,0.0,3.0,168,0,0,0,24.0,1.0,1.0,0.0,1.0


In [65]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

choices = pd.DataFrame()
for j in range(len(test_data)):
    #logically we do want to insert all the stupid columns at once
    test_example = "| " + " ".join([str(row) for row in test_data.iloc[j, :].values])
    choice = vw.predict(test_example) 
    choices = choices.append(pd.DataFrame(choice).T)

In [66]:
choices

,0,1,2,3,4,5,6,7,8,9
0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
0,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0,0.003403,0.003403,0.108464,0.003403,0.596554,0.003403,0.003403,0.003403,0.003403,0.271161
0,0.003403,0.003403,0.542322,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.216929
...,...,...,...,...,...,...,...,...,...,...
0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
0,0.003403,0.003403,0.759250,0.003403,0.108464,0.003403,0.003403,0.003403,0.003403,0.108464
0,0.003403,0.003403,0.542322,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.216929


In [67]:
test_data

,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile
id,,,,,,,,,,,,,,,,,,,,,,
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,1,0,1,0,0.0,0.0,10.0,7.0,168,0,168,0,24.0,1.0,1.0,0.0,1.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,2,0,1,1,10.0,10.0,20.0,17.0,336,510,0,0,36.0,2.0,2.0,1.0,2.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,3,0,2,1,15.0,20.0,30.0,22.0,408,510,0,0,30.0,3.0,3.0,2.0,3.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,4,1,2,1,15.0,20.0,30.0,25.0,504,0,0,0,99.0,4.0,4.0,2.0,4.0
005500a7188546ff8a767329a2f7c76a,0.0,0,54.0,20171209,91000.000000,1,0,0,1,0.0,0.0,10.0,7.0,408,0,408,0,144.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,3,0,2,1,10.0,10.0,30.0,22.0,504,594,0,0,22.0,3.0,3.0,2.0,3.0
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,4,0,3,1,17.0,15.0,35.0,29.0,576,594,0,0,9.0,4.0,3.0,3.0,4.0
fffad4f4828548d1b5583907f2e9906b,0.0,0,54.0,20170123,53399.166667,1,1,0,0,0.0,0.0,0.0,3.0,168,0,0,0,24.0,1.0,1.0,0.0,1.0


In [68]:
train_data_revisited = all_data[all_data.cluster == 2]
test_data_revisited = all_data[all_data.cluster == 1]

In [69]:
coupons_missing_train = train_data_revisited.action.unique()
coupons_missing_train.sort()
coupons_missing_train
# all coupons were served in train

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [70]:
coupons_missing_test = test_data_revisited.action.unique()
coupons_missing_test.sort()
coupons_missing_test
# all coupons were served in test

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [71]:
unique_train, counts_train = np.unique(np.array(train_data_revisited.action), return_counts=True)

In [72]:
print(unique_train)
print(counts_train)
#more or less equally distributed

[0 1 2 3 4 5 6 7 8 9]
[1323 1797 1492 1215 1662 1282 1543 1648 1786 1716]


In [73]:
unique_test, counts_test = np.unique(np.array(test_data_revisited.action), return_counts=True)

In [74]:
print(unique_test)
print(counts_test)
#mehr oder weniger gut gleich verteilt 

[0 1 2 3 4 5 6 7 8 9]
[1704 1666 1594 1739 1566 1718 1621 1607 1675 1633]


In [75]:
#we now have probas per time_step
# now lets see if the proba changes per time step per user or if he would just advise the same every time.

In [76]:
choices.index = test_data.index

In [77]:
for col in choices.columns:
    
    code = coupons_groups[coupons_groups.Encoding == col].coupon_code.unique()
    choices = choices.rename(columns={
        col: "".join(code)
    })

In [78]:
choices

,0b1e1539f2cc45b7b9fa7c272da2e1d7,2298d6c36e964ae4a3e7e9706d1fb8c2,2906b810c7d4411798c6938adc9daaa5,3f207df678b143eea3cee63160fa8bed,4d5c57ea9a6940dd891ad53e9dbe8da0,5a8bc65990b245e5a138643cd4eb9837,9b98b8c7a33c4b65b9aebfe6a799e6d9,ae264e3637204a6fb9bb56bc8210ddfd,f19421c1d4aa40978ebb69ca19b0e20d,fafdcd668e3743c1bb461111dcafc2a4
id,,,,,,,,,,
0020c2b971eb4e9188eac86d93036a77,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
0020c2b971eb4e9188eac86d93036a77,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0020c2b971eb4e9188eac86d93036a77,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0020c2b971eb4e9188eac86d93036a77,0.003403,0.003403,0.108464,0.003403,0.596554,0.003403,0.003403,0.003403,0.003403,0.271161
005500a7188546ff8a767329a2f7c76a,0.003403,0.003403,0.542322,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.216929
...,...,...,...,...,...,...,...,...,...,...
fff7576017104bcc8677a8d63322b5e1,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
fff7576017104bcc8677a8d63322b5e1,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
fffad4f4828548d1b5583907f2e9906b,0.003403,0.003403,0.759250,0.003403,0.108464,0.003403,0.003403,0.003403,0.003403,0.108464


In [79]:
test_data

,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile
id,,,,,,,,,,,,,,,,,,,,,,
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,1,0,1,0,0.0,0.0,10.0,7.0,168,0,168,0,24.0,1.0,1.0,0.0,1.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,2,0,1,1,10.0,10.0,20.0,17.0,336,510,0,0,36.0,2.0,2.0,1.0,2.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,3,0,2,1,15.0,20.0,30.0,22.0,408,510,0,0,30.0,3.0,3.0,2.0,3.0
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,4,1,2,1,15.0,20.0,30.0,25.0,504,0,0,0,99.0,4.0,4.0,2.0,4.0
005500a7188546ff8a767329a2f7c76a,0.0,0,54.0,20171209,91000.000000,1,0,0,1,0.0,0.0,10.0,7.0,408,0,408,0,144.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,3,0,2,1,10.0,10.0,30.0,22.0,504,594,0,0,22.0,3.0,3.0,2.0,3.0
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,4,0,3,1,17.0,15.0,35.0,29.0,576,594,0,0,9.0,4.0,3.0,3.0,4.0
fffad4f4828548d1b5583907f2e9906b,0.0,0,54.0,20170123,53399.166667,1,1,0,0,0.0,0.0,0.0,3.0,168,0,0,0,24.0,1.0,1.0,0.0,1.0


In [80]:
concatenated = pd.concat([test_data, choices], axis=1)

In [81]:
concatenated

,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile,0b1e1539f2cc45b7b9fa7c272da2e1d7,2298d6c36e964ae4a3e7e9706d1fb8c2,2906b810c7d4411798c6938adc9daaa5,3f207df678b143eea3cee63160fa8bed,4d5c57ea9a6940dd891ad53e9dbe8da0,5a8bc65990b245e5a138643cd4eb9837,9b98b8c7a33c4b65b9aebfe6a799e6d9,ae264e3637204a6fb9bb56bc8210ddfd,f19421c1d4aa40978ebb69ca19b0e20d,fafdcd668e3743c1bb461111dcafc2a4
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,1,0,1,0,0.0,0.0,10.0,7.0,168,0,168,0,24.0,1.0,1.0,0.0,1.0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,2,0,1,1,10.0,10.0,20.0,17.0,336,510,0,0,36.0,2.0,2.0,1.0,2.0,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,3,0,2,1,15.0,20.0,30.0,22.0,408,510,0,0,30.0,3.0,3.0,2.0,3.0,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.000000,4,1,2,1,15.0,20.0,30.0,25.0,504,0,0,0,99.0,4.0,4.0,2.0,4.0,0.003403,0.003403,0.108464,0.003403,0.596554,0.003403,0.003403,0.003403,0.003403,0.271161
005500a7188546ff8a767329a2f7c76a,0.0,0,54.0,20171209,91000.000000,1,0,0,1,0.0,0.0,10.0,7.0,408,0,408,0,144.0,1.0,0.0,1.0,1.0,0.003403,0.003403,0.542322,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.216929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,3,0,2,1,10.0,10.0,30.0,22.0,504,594,0,0,22.0,3.0,3.0,2.0,3.0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
fff7576017104bcc8677a8d63322b5e1,0.0,0,55.0,20171031,52760.000000,4,0,3,1,17.0,15.0,35.0,29.0,576,594,0,0,9.0,4.0,3.0,3.0,4.0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
fffad4f4828548d1b5583907f2e9906b,0.0,0,54.0,20170123,53399.166667,1,1,0,0,0.0,0.0,0.0,3.0,168,0,0,0,24.0,1.0,1.0,0.0,1.0,0.003403,0.003403,0.759250,0.003403,0.108464,0.003403,0.003403,0.003403,0.003403,0.108464


In [82]:
#Are there user who got a coupon twice (and if yes, really fulfilled the user journey)?
varying_coupon_proba = pd.DataFrame() 
for user_id in concatenated.index.unique():
    data = concatenated[concatenated.index == user_id].iloc[:, -10:]
    if data.duplicated(keep=False).any() == False:
        varying_coupon_proba = varying_coupon_proba.append(data)

In [90]:
concatenated[concatenated.index=="0020c2b971eb4e9188eac86d93036a77"]

,value,gender,age,became_member_on,income,offer_views_missed,informational,bogo,discount,total_duration_completed,total_difficulty_completed,difficulty_offered,duration_offered,last_time_offer_received,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,mean_time_between_actions,email,social,web,mobile,0b1e1539f2cc45b7b9fa7c272da2e1d7,2298d6c36e964ae4a3e7e9706d1fb8c2,2906b810c7d4411798c6938adc9daaa5,3f207df678b143eea3cee63160fa8bed,4d5c57ea9a6940dd891ad53e9dbe8da0,5a8bc65990b245e5a138643cd4eb9837,9b98b8c7a33c4b65b9aebfe6a799e6d9,ae264e3637204a6fb9bb56bc8210ddfd,f19421c1d4aa40978ebb69ca19b0e20d,fafdcd668e3743c1bb461111dcafc2a4
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.0,1,0,1,0,0.0,0.0,10.0,7.0,168,0,168,0,24.0,1.0,1.0,0.0,1.0,0.003403,0.003403,0.216929,0.003403,0.216929,0.003403,0.003403,0.003403,0.003403,0.542322
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.0,2,0,1,1,10.0,10.0,20.0,17.0,336,510,0,0,36.0,2.0,2.0,1.0,2.0,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.0,3,0,2,1,15.0,20.0,30.0,22.0,408,510,0,0,30.0,3.0,3.0,2.0,3.0,0.003403,0.003403,0.216929,0.003403,0.542322,0.003403,0.003403,0.003403,0.003403,0.216929
0020c2b971eb4e9188eac86d93036a77,0.0,1,24.0,20160304,60000.0,4,1,2,1,15.0,20.0,30.0,25.0,504,0,0,0,99.0,4.0,4.0,2.0,4.0,0.003403,0.003403,0.108464,0.003403,0.596554,0.003403,0.003403,0.003403,0.003403,0.271161


In [88]:
unique, counts = np.unique(choices, return_counts=True)
unique

array([0.00340305, 0.10846428, 0.10846429, 0.21692859, 0.27116072,
       0.27116075, 0.37962502, 0.43385717, 0.5423215 , 0.59655362,
       0.59655368, 0.75925004])

In [85]:
#ansonsten ändert sich die Wahrscheinlichkeit nicht
user_share_test = len(varying_coupon_proba.index.unique())
all_users_test = len(test_data.index.unique())

user_share_test/all_users_test
# for 434 user  the recommendation will change during the time

0.1250276121051469